In [ ]:
import pandas as pd
import requests
import re
import os

# **Use #reviewkhachsan193.csv to get required column**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/#reviewkhachsan193.csv")

In [ ]:
df.head(2)

,authorMeta/digg,authorMeta/fans,authorMeta/heart,webVideoUrl,isAd,isSponsored,playCount,diggCount,commentCount,shareCount,videoMeta/duration,videoMeta/subtitleLinks/1/downloadLink,text
0,42,33,2553,https://www.tiktok.com/@coinecoinee/video/7502...,False,False,907,42,2,5,132,https://api.apify.com/v2/key-value-stores/Qivr...,Review khách sạn view triệu đô giữa lòng Bangk...
1,1969,11500,215000,https://www.tiktok.com/@tony_khoa/video/740338...,False,False,10300,36,3,35,60,https://api.apify.com/v2/key-value-stores/Qivr...,Midas hotel khách sạn đẹp giữa trung tâm phú n...


In [ ]:
df.columns.to_list()

['authorMeta/digg',
 'authorMeta/fans',
 'authorMeta/heart',
 'webVideoUrl',
 'isAd',
 'isSponsored',
 'playCount',
 'diggCount',
 'commentCount',
 'shareCount',
 'videoMeta/duration',
 'videoMeta/subtitleLinks/1/downloadLink',
 'text']

# **Process all files**



In [ ]:
# Đường dẫn tới thư mục chứa các tệp CSV
folder_path = '/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/excel/'

# Danh sách các cột cần giữ lại
columns_to_keep = [
    'authorMeta/digg',
    'authorMeta/fans',
    'authorMeta/heart',
    'webVideoUrl',
    'isAd',
    'isSponsored',
    'playCount',
    'diggCount',
    'commentCount',
    'shareCount',
    'videoMeta/duration',
    'videoMeta/subtitleLinks/1/downloadLink',
    'text'
]

# Lấy danh sách tất cả các tệp CSV trong thư mục
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Nếu có tệp CSV hợp lệ trong thư mục
if csv_files:
    # Danh sách lưu trữ các DataFrame đã xử lý
    processed_data = []

    # Duyệt qua tất cả các tệp CSV
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        # Đọc tệp CSV vào DataFrame
        df = pd.read_csv(file_path)

        # Kiểm tra nếu cột videoMeta/subtitleLinks/1/downloadLink có trong DataFrame
        if 'videoMeta/subtitleLinks/1/downloadLink' in df.columns:
            # Chỉ giữ lại các cột yêu cầu
            df = df[columns_to_keep]

            # Lọc các dòng không có giá trị null ở cột videoMeta/subtitleLinks/1/downloadLink
            df = df[df['videoMeta/subtitleLinks/1/downloadLink'].notnull()]

            # Sắp xếp các cột theo đúng thứ tự yêu cầu
            df = df[columns_to_keep]

            # In tên tệp đã đọc
            print(f"Đã đọc tệp: {csv_file}")

            # Thêm DataFrame đã xử lý vào danh sách
            processed_data.append(df)

    # Nếu có dữ liệu đã được xử lý, kết hợp tất cả vào một DataFrame duy nhất
    if processed_data:
        final_df = pd.concat(processed_data, ignore_index=True)

        # Hiển thị một số dòng đầu tiên của kết quả
        print(final_df.head(2))

        # Nếu muốn lưu kết quả vào một tệp CSV mới
        # final_df.to_csv('/content/drive/MyDrive/output_file.csv', index=False)
    else:
        print("Không có dữ liệu hợp lệ nào để xử lý.")
else:
    print("Không tìm thấy tệp CSV hợp lệ trong thư mục.")


Đã đọc tệp: reviewhotel189.csv
Đã đọc tệp: reviewhotel209.csv
Đã đọc tệp: hotelsaigon190.csv
Đã đọc tệp: hotelhanoi234.csv
Đã đọc tệp: hotelhalong152.csv
Đã đọc tệp: hotelvungtau158.csv
Đã đọc tệp: hotelnhatrang262.csv
Đã đọc tệp: hotelluxury207.csv
Đã đọc tệp: hotelquan1-184.csv
Đã đọc tệp: hotelcantho210.csv
Đã đọc tệp: hoteldalat265.csv
Đã đọc tệp: hotelgiare209.csv
Đã đọc tệp: hotelphuquy170.csv
Đã đọc tệp: hotelphuquoc234.csv
Đã đọc tệp: hoteldanang159.csv
Đã đọc tệp: hotelhue234.csv
Đã đọc tệp: hoteldaklak100.csv
Đã đọc tệp: hotelhoian234.csv
Đã đọc tệp: hotelquangbinh238.csv
Đã đọc tệp: hotelninhthuan260.csv
Đã đọc tệp: hotelhaiphong214.csv
Đã đọc tệp: hotelhagiang230.csv
Đã đọc tệp: hoteltuyenquang210.csv
Đã đọc tệp: #khachsan180lastest.csv
Đã đọc tệp: review_resort3.csv
Đã đọc tệp: homestay_oldest.csv
Đã đọc tệp: review_resort2.csv
Đã đọc tệp: reviewhomestay_latest.csv
Đã đọc tệp: homestay_latest.csv
Đã đọc tệp: #khachsan162.csv
Đã đọc tệp: review_resort1.csv
Đã đọc tệp: revie

In [ ]:
final_df = final_df.drop_duplicates(subset='webVideoUrl', keep='first')

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1281 entries, 0 to 2305
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   authorMeta/digg                         1281 non-null   int64 
 1   authorMeta/fans                         1281 non-null   int64 
 2   authorMeta/heart                        1281 non-null   int64 
 3   webVideoUrl                             1281 non-null   object
 4   isAd                                    1281 non-null   bool  
 5   isSponsored                             1281 non-null   bool  
 6   playCount                               1281 non-null   int64 
 7   diggCount                               1281 non-null   int64 
 8   commentCount                            1281 non-null   int64 
 9   shareCount                              1281 non-null   int64 
 10  videoMeta/duration                      1281 non-null   int64 
 11  videoMeta

# **Change script link suffix to pattern -vie-VN**

In [ ]:
def fix_subtitle_link(url):
    if not url.endswith('-vie-VN'):
        url = url.rsplit('-', 1)[0] + '-vie-VN'
    return url

final_df['videoMeta/subtitleLinks/1/downloadLink'] = final_df['videoMeta/subtitleLinks/1/downloadLink'].apply(fix_subtitle_link)

In [ ]:
final_df.insert(0, 'id', ['video' + str(i).zfill(2) for i in range(1, len(final_df) + 1)])


<bound method NDFrame.head of              id  authorMeta/digg  authorMeta/fans  authorMeta/heart  \
0       video01             8861            32300           1000000   
1       video02            14800           169500           5300000   
2       video03            16700           200400           7300000   
3       video04            43100            21500            477000   
4       video05            13800           296600           6700000   
...         ...              ...              ...               ...   
2296  video1277              213             3218             78300   
2298  video1278               72             1246             18000   
2303  video1279             8420           172100           3300000   
2304  video1280              644            25400            313400   
2305  video1281              255             8586             13200   

                                            webVideoUrl   isAd  isSponsored  \
0     https://www.tiktok.com/@dannie.l

In [ ]:
final_df['videoMeta/subtitleLinks/1/downloadLink'].head(10)

,videoMeta/subtitleLinks/1/downloadLink
0,https://api.apify.com/v2/key-value-stores/cfrJ...
1,https://api.apify.com/v2/key-value-stores/cfrJ...
2,https://api.apify.com/v2/key-value-stores/cfrJ...
3,https://api.apify.com/v2/key-value-stores/cfrJ...
4,https://api.apify.com/v2/key-value-stores/cfrJ...
5,https://api.apify.com/v2/key-value-stores/cfrJ...
6,https://api.apify.com/v2/key-value-stores/cfrJ...
7,https://api.apify.com/v2/key-value-stores/cfrJ...
8,https://api.apify.com/v2/key-value-stores/cfrJ...
9,https://api.apify.com/v2/key-value-stores/cfrJ...


In [ ]:
final_df.to_csv("/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/#merge_test1281.csv")

# **Extract script video**

In [ ]:
final_df['script_video'] = ''

In [ ]:
output_folder = "/content/drive/MyDrive/pizza/CÔNG NGHỆ ĐỔI MỚI (1)/CUỐI KỲ/UP file csv lên/script/"

for index, row in final_df.iterrows():
    video_id = row['id']
    txt_filename = os.path.join(output_folder, f"{video_id}.txt")

    try:
        with open(txt_filename, "r", encoding="utf-8") as f:
            script_content = f.read()
            final_df.loc[index, 'script_video'] = script_content
    except FileNotFoundError:
        print(f"Không tìm thấy tệp script cho video_id: {video_id}")

In [ ]:
final_df['script_video'].iloc[190]

'với giá chưa đến 300 k\nkhách sạn 4 SAO ở Nha Trang view biển bình Minh xinh cỡ này\nNha Trang biển đẹp\nđồ ăn lại ngon mà giá khách sạn cũng rẻ quá trời luôn\nphòng tui ở là khách sạn sania tầng 18\nsăn được qua chevroletca có a mã giảm giá\nvới lại do tui đi ngày thường\nnên phòng này tui thuê chỉ có giá hơn 500 k thôi\nchia Ra chưa đến 300 k một người hạng\nphòng tui để ở trên video để mấy bà tham khảo nha\nkhách sạn còn có buffet sáng bể bơi\nphòng tập gym nữa đó\nnhưng mà hôm tui đi quên mất\nnên không có trải nghiệm hết được ưng\nnhất phải kể đến cái view biển lúc bình Minh\ntrời ơi nó xinh xỉu luôn í\ncỡ này rất phù hợp với những bà thích đi du lịch\nhưu Linh nhưng lại ngại Bon Chen ngoài đường giống như tui'